In [1]:
import pandas as pd;
import numpy as np;
import matplotlib.pyplot as plt;
import glob as glob

In [2]:
### get files
kk_arr = glob.glob('D:/Vasiliev/PERMAK-A/nv/8d9d/common/*.csv', recursive=True)
kq_arr = glob.glob('D:/Vasiliev/PERMAK-A/nv/8d9d/common/*.txt', recursive=True)

In [3]:
### Calculate max
def GetMaxInitKq():
    kq_init=pd.read_csv('~D:/Vasiliev/PERMAK-A/nv/8d9d/common/z9-01.txt',names=['kq']);
    cycle_length=26;
    kq_init['time']=1;
    kq_init['tvs']=-1;
    inx=0;
    cycle_lenght = 26;
    for _time in range(cycle_length):
        for _tvs in range(163):
            kq_init.loc[inx,'time'] = _time;
            kq_init.loc[inx,'tvs'] = _tvs+1;
            inx+=1;

    kqmax=kq_init.query('time==0').sort_values('kq').tail(20)
    len(np.unique(kqmax.tvs))
    return kqmax.tvs.tolist();

In [4]:
### Get KK values
def GetDatasetKK(_folder,state_indexer):
    dfa=[];
    for i in _folder:
        dataset_kk=pd.read_csv(i ,names=['tvs','tvel','time','kk'],sep=';' )
        dataset_kk.tvel+=1
        dataset_kk['state']=state_indexer;
        state_indexer+=1;
        #dt_kk=dataset_kk.query('tvel==272 or tvel==282 or tvel==292 or tvel==302 or tvel==312 or tvel==322')
        dt_kk=dataset_kk.query('tvel==277 or tvel==287 or tvel==297 or tvel==307 or tvel==317 or tvel==327')
        dt_kk=pd.concat([dt_kk.query('tvs==@i') for i in (kq_max_arr)], ignore_index=True, sort=False)
        dfa.append(dt_kk);
    df=pd.concat(dfa, ignore_index=True, sort=False);
    print('folder ready!');
    return df,state_indexer;

In [5]:
### Get KQ values
def GetDatasetKQ(_folder,state_indexer):
    dfa=[]
    for i in _folder:
        dataset_kq=pd.read_csv(i, names=['kq'])
        dataset_kq['time']=1;
        dataset_kq['tvs']=-1;
        dataset_kq['state']=state_indexer;
        state_indexer+=1;
        inx=0;
        cycle_length = 26;
        for _time in range(cycle_length):
            for _tvs in range(163):
                dataset_kq.loc[inx,'time'] = _time;
                dataset_kq.loc[inx,'tvs'] = _tvs+1;
                inx+=1;
        dt_kq = pd.concat([dataset_kq.query('tvs == @i') for i in kq_max_arr], ignore_index=True, sort=False);
        dfa.append(dt_kq);
    df=pd.concat(dfa, ignore_index=True, sort=False);
    print('folder ready');
    return df,state_indexer;

In [6]:
## Dataset eval
def Dataset(dataset_kq, dataset_kk):
    kr=[]
    kq=[]
    for i in range(len(dataset_kk)):
        val=dataset_kk.iloc[i];
        kq_val=dataset_kq.query('state==@val.state & tvs==@val.tvs')
        kq_val.index=np.arange(0,len(kq_val), step = 1);
        kr.append(val.kk*kq_val.loc[0,'kq'])
        kq.append(kq_val.loc[0,'kq'])
    dataset_kk['kq']=kq
    dataset_kk['kr']=kr
    print('state ready')
    return dataset_kk;

In [7]:
state_inx = 0;
kq_max_arr = GetMaxInitKq();
dataset_kq_prv,state_indexer=GetDatasetKQ(kq_arr, state_inx)
state_indexer=0;
dataset_kk_prev,state_indexer=GetDatasetKK(kk_arr,state_indexer)

folder ready
folder ready!


In [8]:
dataset_kq = pd.concat([dataset_kq_prv.query('time == @tm') for tm in [0,13,25]], ignore_index=True, sort=False)
dataset_kk = pd.concat([dataset_kk_prev.query('time == @tm') for tm in [0,13,25]], ignore_index=True, sort=False)

In [9]:
united_b = Dataset(dataset_kq.query('time==0'), dataset_kk.query('time==0'));
united_m = Dataset(dataset_kq.query('time==13'), dataset_kk.query('time==13'));
united_e = Dataset(dataset_kq.query('time==25'), dataset_kk.query('time==25'));

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


state ready
state ready
state ready


In [10]:
united_b.to_csv('D:/Vasiliev/PERMAK-A/nv/8d9d/s_cen_b.csv', index=False)
united_m.to_csv('D:/Vasiliev/PERMAK-A/nv/8d9d/s_cen_m.csv',index=False)
united_e.to_csv('D:/Vasiliev/PERMAK-A/nv/8d9d/s_cen_e.csv',index=False)